In [1]:
from PIL import Image
import numpy
import sys
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
import random
import os
import comtypes.client
from docx2pdf import convert
from pdf2image import convert_from_path
from docx.enum.style import WD_STYLE_TYPE

In [3]:
def find_coeffs(pa, pb):
    matrix = []
    for p1, p2 in zip(pa, pb):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

    A = numpy.matrix(matrix, dtype=numpy.float)
    B = numpy.array(pb).reshape(8)

    res = numpy.dot(numpy.linalg.inv(A.T * A) * A.T, B)
    return numpy.array(res).reshape(8)

def letterToMadness(letter):
    if(letter == 'a'):
        return random.choice(['a', '╞','╟','┼'])
    if(letter == 'b'):
        return random.choice(['b','⅝','╚'])
    if(letter == 'c' or letter == 'č' or letter == 'ć'):
        return random.choice(['c','╩','╔'])
    if(letter == 'd' or letter == 'đ'):
        return random.choice(['d','╦','╠'])
    if(letter == 'e'):
        return random.choice(['e','†','≤','═','╧','╬'])
    if(letter == 'f'):
        return random.choice(['f','╨','╤'])
    if(letter == 'g'):
        return random.choice(['g','℁','⋲','╥'])
    if(letter == 'h'):
        return random.choice(['h','╒','╘'])
    if(letter == 'i'):
        return random.choice(['i','╪','╫','╓'])
    if(letter == 'j'):
        return random.choice(['j','┘'])
    if(letter == 'k'):
        return random.choice(['k','┌'])
    if(letter == 'l'):
        return random.choice(['l','℅','▄','█'])
    if(letter == 'm'):
        return random.choice(['m','←','▌','▐'])
    if(letter == 'n'):
        return random.choice(['n','▀','☻'])
    if(letter == 'o'):
        return random.choice(['o','∂','♥','♦','₧','‡'])
    if(letter == 'p'):
        return random.choice(['p','‰'])
    if(letter == 'q'):
        return random.choice(['q'])
    if(letter == 'r'):
        return random.choice(['r','◘','○','╡'])
    if(letter == 's' or letter == 'š'):
        return random.choice(['s','◙','♂','♀','╢'])
    if(letter == 't'):
        return random.choice(['t','♪','♫','☼','≡'])
    if(letter == 'u'):
        return random.choice(['u','►','◄'])
    if(letter == 'v'):
        return random.choice(['v','↕','◄'])
    if(letter == 'w'):
        return random.choice(['w','‹','‼'])
    if(letter == 'x'):
        return random.choice(['x','›'])
    if(letter == 'y'):
        return random.choice(['y','Ⅎ','≠'])
    if(letter == 'z' or letter == 'ž'):
        return random.choice(['z','→'])
    if(letter == '-'):
        return random.choice(['‐','–','—'])
    if(letter == 'Č' or letter == 'Ć'):
        return 'C'
    if(letter == 'Š'):
        return 'S'
    if(letter == 'Đ'):
        return 'D'
    if(letter == 'Ž'):
        return 'Z'
    return letter
    

In [14]:
document = Document('example.docx')


# Remove Header and Footer
for section in document.sections:
    header = section.header
    footer = section.footer
    header.is_linked_to_previous = True
    footer.is_linked_to_previous = True



for elem in document.element.getiterator():
    if("CT_Text" in str(type(elem)) and elem.text):
        newt = ""
        for ch in elem.text:
            newt += letterToMadness(ch)
        elem.text = newt
        


for stylex in document.styles:
    try:
        stylex.font.name = 'PremiumUltra17'
    except:
        pass

for para in document.paragraphs:
    print(para.style.name)
    para.style.font.name = 'PremiumUltra17'
    for run in para.runs:
        run.font.name = 'PremiumUltra17'

document.save('scriptalized.docx')


Title
Normal
Normal
Heading 1
Normal
Normal
Normal
Normal
Heading 1
Heading 1
Normal
Normal
Normal


In [7]:
convert('scriptalized.docx')


  0%|          | 0/1 [00:00<?, ?it/s]

com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Word', 'Command failed', 'wdmain11.chm', 36966, -2146824090), None)

In [4]:


pages = convert_from_path('scriptalized.pdf', 300, poppler_path = r"D:\Marko\poppler-21.03.0\Library\bin")

i=0
for page in pages:
    i+=1
    page.save('tmp/page'+str(i)+'.jpg', 'JPEG')
x=0
while x < i:

    img = Image.open("book1.jpg")
    foreground = Image.open("tmp/page"+str(x+1)+".jpg")

    width, height = foreground.size
    bigW, bigH = img.size
    coeffs = find_coeffs(
            [(96, 226), (1660, 166), (1735, 2449), (141, 2478)],
            [(0, 0), (width, 0), (width, height), (0, height)])
    foreground.putalpha(255)

    datas = foreground.getdata()

    newData = []
    for item in datas:
        if item[0] > 0 and item[1] > 0 and item[2] > 0:
            alfavalue = int(255 - (item[0]+item[1]+item[2])/3)
            newData.append((item[0], item[1], item[2], alfavalue))
        else:
            newData.append(item)

    foreground.putdata(newData)


    foreground = foreground.transform((bigW, bigH), Image.PERSPECTIVE, coeffs, Image.BICUBIC)
    img.paste(foreground, (0, 0), foreground.convert('RGBA'))

    if x + 1 < i:
        foreground = Image.open("tmp/page"+str(x+2)+".jpg")
        width, height = foreground.size
        bigW, bigH = img.size
        coeffs = find_coeffs(
                [(1672, 145), (3197, 176), (3279, 2376), (1732, 2470)],
                [(0, 0), (width, 0), (width, height), (0, height)])
        foreground.putalpha(255)

        datas = foreground.getdata()

        newData = []
        for item in datas:
            if item[0] > 0 and item[1] > 0 and item[2] > 0:
                alfavalue = int(255 - (item[0]+item[1]+item[2])/3)
                newData.append((item[0], item[1], item[2], alfavalue))
            else:
                newData.append(item)

        foreground.putdata(newData)


        foreground = foreground.transform((bigW, bigH), Image.PERSPECTIVE, coeffs, Image.BICUBIC)
        img.paste(foreground, (0, 0), foreground.convert('RGBA'))

    img.save("output/Marko Pekas "+str(x+1)+"-"+str(x+2)+".png", "PNG")

    x+=2

  0%|          | 0/1 [00:00<?, ?it/s]

com_error: (-2147352567, 'Exception occurred.', (0, 'Microsoft Word', 'Command failed', 'wdmain11.chm', 36966, -2146824090), None)